In [1]:
import pandas as pd
import numpy as np

from core.utils import full_file_path

In [2]:
np.random.seed(42)

In [3]:
path_dados = full_file_path('dados_merged.csv', 'generated_data')
df = pd.read_csv(path_dados, sep = ';', encoding='utf-8')

C:\Users\d835916\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,Unnamed: 0,Data do Pedido,Número de Controle,Forma do Pedido,Orgão/Entidade,SIC,Pedido,Síntese,Forma de recebimento da resposta,Situação do pedido,...,Anexo Resposta,Tipo Pessoa,Sexo,Data Nascimento,Município,Estado,País,Escolaridade,Profissão,_merge
0,0,NaN,56725811650,NaN,Secretaria de Governo,Central de Atendimento ao Cidadão - CAC,Solicito meu atestado ou certidão de tempo de ...,Solicito meu atestado ou certidão de tempo de ...,Por carta,Encerrada,...,NaN,PF,NaN,NaN,Oriente,São Paulo,Brasil,Ensino superior,Servidor público municipal,both
1,1,NaN,94006811820,NaN,Secretaria Estadual de Gestão Pública,< Outros - Gestão Pública >,como conseguir tirar um novo rg,como conseguir tirar um novo rg,Consulta pelo sistema,Encerrada,...,NaN,PF,NaN,NaN,São Paulo,São Paulo,Brasil,NaN,NaN,both
2,2,NaN,927607102560,E-mail,Departamento de Trânsito - DETRAN,Departamento de Trânsito - DETRAN,"Prezado Sr., Faço uma pesquisa para ser utiliz...","Prezado Sr., Faço uma pesquisa para ser utiliz...",Correspondência eletrônica (e-mail),Encerrada,...,resultado.xlsx,PF,NaN,NaN,São Paulo,São Paulo,Brasil,NaN,NaN,both
3,3,NaN,66342711990,NaN,Secretaria Estadual de Gestão Pública,< Outros - Gestão Pública >,teste,teste,Buscar/Consultar pessoalmente,Encerrada,...,NaN,PF,NaN,NaN,NaN,NaN,Brasil,NaN,NaN,both
4,4,NaN,66670812160,NaN,Secretaria Estadual de Gestão Pública,< Outros - Gestão Pública >,Como posso fazer se eu quiser enviar para uma ...,Como posso fazer se eu quiser enviar para uma ...,Buscar/Consultar pessoalmente,Encerrada,...,NaN,PF,NaN,NaN,NaN,NaN,Brasil,NaN,NaN,both


In [5]:
def padronizar_nom_coluna(nom):
    
    nom = nom.lower()
    nom = nom.replace(' ', '_')
    
    return nom

nomes_cols = {col : padronizar_nom_coluna(col) 
             for col in df.columns}
df = df.rename(nomes_cols, axis = 1)


In [6]:
for col in df.columns:
    print('\n')

    print(f'{col} : {df[col].dtype}')
    print(f'Nulls: {df[col].isnull().mean()}')
    
    if df[col].dtype == 'O':
        print(f'Categorias: {df[col].nunique()}')
    else:
        print(f'Max: {df[col].max()}')
        print(f'Min: {df[col].min()}')
    print(df[col].sample())
    print('*'*100)



unnamed:_0 : int64
Nulls: 0.0
Max: 190642
Min: 0
61433    61433
Name: unnamed:_0, dtype: int64
****************************************************************************************************


data_do_pedido : object
Nulls: 0.9999685275619876
Categorias: 6
25338    NaN
Name: data_do_pedido, dtype: object
****************************************************************************************************


número_de_controle : int64
Nulls: 0.0
Max: 147919060913910
Min: 170
154882    89250342831180
Name: número_de_controle, dtype: int64
****************************************************************************************************


forma_do_pedido : object
Nulls: 0.00015736219006205315
Categorias: 6
176386    Formulário Eletrônico
Name: forma_do_pedido, dtype: object
****************************************************************************************************


orgão/entidade : object
Nulls: 0.0
Categorias: 104
153196    Departamento de Trânsito - DETRAN
Name: orgão/e

In [7]:
colunas_muitos_nulos = {}

for col in df.columns:
    prop_null = df[col].isnull().mean()
    if  prop_null >= 0.9:
        colunas_muitos_nulos[col] = prop_null
        
    
print(colunas_muitos_nulos)

df = df.drop(list(colunas_muitos_nulos.keys()), axis=1)
        

{'data_do_pedido': 0.9999685275619876, 'data_da_resposta': 0.9999213189049689, 'data_nascimento': 0.9997534659022361}


In [8]:
colunas_id = [
    'unnamed:_0', 
    'número_de_controle',
        ]

df = df.drop(colunas_id, axis=1).copy().reset_index(drop=True)



In [9]:
df['secretaria_governo'] = df['orgão/entidade']=='Secretaria de Governo'

df = df.drop('orgão/entidade', axis=1)

In [10]:
df['capital'] = df['município']=='São Paulo'


cidades_mais_quinhentos = [
                        'Campinas', 
                        'Guarulhos', 
                        'Sorocaba',
                        'São Bernardo do Campo',
                        'São José dos Campos',
                        'Santo André',
                        'Ribeirão Preto',
                        'Osasco',
                    ]
df['cidade_grande'] = df['município'].isin(cidades_mais_quinhentos)

In [11]:
set(df[df['cidade_grande']]['município'].unique()) == set(cidades_mais_quinhentos)

True

In [12]:
df = df.drop('município', axis=1)

In [13]:
df['país'] = df['país'].str.lower()

In [14]:
grafias_brasil_ou_lixo = {
    'brásil',
    'ブラジル', # é Brasil em japonês
    'bras il',
    'bbrasil',
    'brasl',
    'leme',
    'piracicaba',
    'rua santa virginia',
    'avnida lagedao n. 369 cidadessoberana',
    'barasil',
    'bradil',
    'brasile',
    'brésil',
    'campinas brasil',
    'brasíl',
    'brasileira',
    'brasileira',
    'wbrasil',
    'regina vieira de oliveira',
    'brasilbrasil',
    'são paulo',
    'brazil',
    'brasilsp',
    'br',
    'bb',
    'bbbb',
    'sp',
    'sao paulo',
    'brazileiro',
    'brasilbrasilbrasilbrasilbrasil',
    'brasileiro',
    '12970-000', #eh um cep do brasil
    'brasilien',
    "'",
    'brasilz',
    'brasi',
    'ferraz de vasconcelos',
    'brasileiros',
    'brasil e',
    'l',
    'bbbbbb',
    'kib', 
    '🎶🎶👌👌',
       
}

df['país'] = df['país'].apply(lambda x: 'brasil' if x in grafias_brasil_ou_lixo else x)

In [15]:
df['estrangeiro'] = df['país'].apply(lambda x: x!='brasil' and not pd.isnull(x))

df = df.drop('país', axis=1)

In [16]:
df['reside_fora_sp'] = df['estado']!='São Paulo'

df = df.drop('estado', axis=1)

In [17]:
df['profissão'].value_counts(dropna=False)

NaN                              70983
Outra                            27003
Servidor público estadual        19669
Empregado - Setor privado        17180
Estudante                        10669
Profissional liberal/Autônomo    10296
Professor                         7632
Jornalista                        7166
Servidor público municipal        5692
Empresário/Empreendedor           5502
Pesquisador                       4780
Servidor público federal          2625
Membro de ONG nacional             791
Membro de partido político         411
Representante de sindicato         182
Membro de ONG internacional         62
Name: profissão, dtype: int64

In [18]:
df['profissao_terceiro_setor'] = df['profissão'].isin({'Membro de ONG internacional', 'Membro de ONG nacional'})
df['profissao_jornalista'] = df['profissão']=='Jornalista'
df['profissao_academia'] = df['profissão'].isin({'Estudante', 'Professor', 'Pesquisador'})
df['profissao_partido_politico'] = df['profissão'] == 'Membro de partido político'
df['profissao_servidor_publico'] = df['profissão'].isin({'Servidor público municipal', 'Servidor público estadual', 'Servidor público federal'})
df['profissao_sindicalista'] = df['profissão']=='Representante de sindicato'
df['profissao_empresario'] = df['profissão']=='Empresário/Empreendedor'

In [19]:
df = df.drop('profissão',axis=1)

In [20]:
df['forma_do_pedido'].value_counts()

Formulário Eletrônico    186514
E-mail                     2122
Dispositivo Movel          1115
Carta                       417
Pessoalmente                380
Telefone                     65
Name: forma_do_pedido, dtype: int64

In [21]:
df['pedido_pessoal'] = df['forma_do_pedido'].isin({'Pessoalmente', 'Telefone', 'Carta'})

In [22]:
df = df.drop('forma_do_pedido', axis=1)

In [23]:
df['mulher'] = df['sexo']=='F'
df = df.drop('sexo', axis=1)

In [24]:
# a dummy é contra a escolaridade nao informada!

df_escolaridade = pd.get_dummies(df['escolaridade'])
df_escolaridade = df_escolaridade.rename({escol : f'escol_{escol.lower().replace(" ", "_")}'
                                          for escol in df_escolaridade}, axis=1)

In [25]:
df = pd.concat([df, df_escolaridade], axis=1)

In [26]:
df = df.drop('escolaridade', axis=1)

In [27]:
del df_escolaridade

In [28]:
df['natureza_da_solicitação'].value_counts()

#isso é analise após a solicitacao - é pós-fato

df = df.drop('natureza_da_solicitação', axis=1)

FILTRANDO OS DADOS PARA APENAS PESSOA FISICA, COM SOLICITACAO ENCERRADA

In [29]:
df['tipo_pessoa'].value_counts()

PF    170289
PJ     20354
Name: tipo_pessoa, dtype: int64

In [30]:
pessoa_juridica = df['tipo_pessoa']=='PJ'

df = df[~pessoa_juridica].copy().reset_index(drop=True)

df = df.drop('tipo_pessoa', axis=1)

In [31]:
df['situação_do_pedido'].value_counts()

Encerrada                      167858
Resposta enviada ao cidadão      1724
Recebida                          260
Em Análise                        257
Em Recurso                        188
Respondida                          2
Name: situação_do_pedido, dtype: int64

In [32]:
encerrados = df['situação_do_pedido'] == 'Encerrada'

df = df[encerrados].copy().reset_index(drop=True)

In [33]:
df = df.drop('situação_do_pedido', axis=1)

In [34]:
df['a_solicitação_foi'].value_counts()

Atendida                 149866
Atendida Parcialmente      8961
Negada                     7120
Não Respondido                1
Name: a_solicitação_foi, dtype: int64

In [35]:
nao_atendidas = ['Negada', 'Não Respondido']
nao_atendidas = df['a_solicitação_foi'].isin(nao_atendidas)
df['nao_atendidas'] = nao_atendidas

negadas = df[nao_atendidas].copy().reset_index(drop=True)
atendidas = df[~nao_atendidas].copy().reset_index(drop=True)

amostra_atendidas = atendidas.sample(len(negadas), random_state=42)

In [36]:
df = pd.concat([negadas, amostra_atendidas])

df['nao_atendidas'].value_counts()

True     7121
False    7121
Name: nao_atendidas, dtype: int64

In [37]:
df = df.drop('a_solicitação_foi', axis=1)

In [38]:
df.columns

Index(['sic', 'pedido', 'síntese', 'forma_de_recebimento_da_resposta',
       'resposta', 'prorrogada', 'redirecionado_por',
       'tempo_resposta_(em_dias)', 'anexo_resposta', '_merge',
       'secretaria_governo', 'capital', 'cidade_grande', 'estrangeiro',
       'reside_fora_sp', 'profissao_terceiro_setor', 'profissao_jornalista',
       'profissao_academia', 'profissao_partido_politico',
       'profissao_servidor_publico', 'profissao_sindicalista',
       'profissao_empresario', 'pedido_pessoal', 'mulher', 'escol_doutorado',
       'escol_ensino_fundamental', 'escol_ensino_médio',
       'escol_ensino_superior', 'escol_especialização', 'escol_mestrado',
       'escol_sem_instrução_formal', 'nao_atendidas'],
      dtype='object')

In [43]:
#inicialmente nao vou considerar o texto na modelagem
#depois podemos fazer uma NLP

colunas_modelagem = [
    'secretaria_governo', 
    'capital', 
    'cidade_grande', 
    'estrangeiro',
    'reside_fora_sp',
    'profissao_terceiro_setor', 
    'profissao_jornalista',
    'profissao_academia', 
    'profissao_partido_politico',
    'profissao_servidor_publico', 
    'profissao_sindicalista',
    'profissao_empresario', 
    'pedido_pessoal', 
    'mulher',
    'escol_doutorado',
   'escol_ensino_fundamental', 
    'escol_ensino_médio',
   'escol_ensino_superior', 
    'escol_especialização', 
    'escol_mestrado',
   'escol_sem_instrução_formal', 
    'nao_atendidas'
    ]

In [44]:
df = df[colunas_modelagem]

In [47]:
path_file = full_file_path('dados_modelagem.csv', 'generated_data')

In [48]:
df.to_csv(path_file, sep=';', encoding='utf-8')